<a href="https://colab.research.google.com/github/khyejin1231/Masters-Thesis/blob/main/Deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
from google.colab import drive
import os
drive.mount('/content/drive', force_remount = True)
PATH = '/content/drive/My Drive/Colab Notebooks/BDS_Thesis'

#!pip install pickle5
#import pickle5 as pickle

#infile = open(PATH+'/final_labels_no_artists_lemma_10.pkl','rb')
#df = pickle.load(infile)


#url = 'https://github.com/khyejin1231/Masters-Thesis/blob/main/final_labels_no_artists_lemma_10.pkl?raw=true'

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/BDS_Thesis/joined_selected_BERT_xartist_lemma10.csv')

df.reset_index(drop=True, inplace=True)

Mounted at /content/drive


In [8]:
df = df.sample(n = 30000)

In [9]:
!pip install requests
!pip install tabulate
!pip install future
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o


Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html


In [10]:
import h2o
h2o.init()

df_h2o = h2o.H2OFrame(df)
train, valid, test = df_h2o.split_frame(ratios = [0.8, 0.1])

y= "Gender_NBinary"
x = train.names[0:768]

train["Gender_NBinary"] = train["Gender_NBinary"].asfactor()
test["Gender_NBinary"] = test["Gender_NBinary"].asfactor()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19; OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpi4rd2u63
  JVM stdout: /tmp/tmpi4rd2u63/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpi4rd2u63/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,"28 days, 15 hours and 35 minutes"
H2O_cluster_name:,H2O_from_python_unknownUser_cuwx14
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
assignment_type = "Stratified"

hidden_opt = [[16,32, 16],[8,16, 8],[32,16,8],[248, 124, 248],[10,10,10,10],[124,62,124]]
hidden_dropout = [[0.2,0.2,0.2],[0.5,0.5,0.5]]
input_dropout_ratio = [0.1, 0.2, 0.5]
activation = ["rectifier_with_dropout", "tanh_with_dropout"]

search_criteria = {"strategy":"RandomDiscrete", "max_models":50, "max_runtime_secs":100,
          "seed":123456}         


hyper_parameters = {"hidden": hidden_opt, "input_dropout_ratio": 
                    input_dropout_ratio, "activation": activation, "hidden_dropout_ratios" : hidden_dropout}

gs_deeplearning = H2OGridSearch(H2ODeepLearningEstimator(
                fast_mode = True, balance_classes = True, variable_importances = False, fold_assignment = assignment_type,
                   nfolds = 5), hyper_params = hyper_parameters, search_criteria = search_criteria, parallelism = 0)

gs_deeplearning.train(x=x, y=y, training_frame=train, validation_frame = valid)
gs_deeplearning.show()


deeplearning Grid Build progress: |██████████████████████████████████████████████| (done) 100%
               activation          hidden hidden_dropout_ratios  \
0    RectifierWithDropout  [124, 62, 124]       [0.5, 0.5, 0.5]   
1    RectifierWithDropout    [16, 32, 16]       [0.5, 0.5, 0.5]   
2    RectifierWithDropout      [8, 16, 8]       [0.5, 0.5, 0.5]   
3    RectifierWithDropout      [8, 16, 8]       [0.5, 0.5, 0.5]   

   input_dropout_ratio                           model_ids   logloss  
0                  0.2  DeepLearning_model_1652339264715_2  0.646923  
1                  0.1  DeepLearning_model_1652339264715_5  0.657952  
2                  0.2  DeepLearning_model_1652339264715_3  0.664421  
3                  0.5  DeepLearning_model_1652339264715_4  0.665873  


In [14]:
grid = gs_deeplearning.get_grid(sort_by='F1', decreasing=True)

In [15]:
grid[0]

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_model_1652339264715_2


Status of Neuron Layers: predicting Gender_NBinary, 2-class classification, bernoulli distribution, CrossEntropy loss, 111,168 weights/biases, 1.4 MB, 282,640 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,768,Input,20.0,,,,,,,,,
1,,2,124,RectifierDropout,50.0,0.0,0.0,0.023557,0.007531,0.0,0.000543,0.072399,0.35705,0.11221
2,,3,62,RectifierDropout,50.0,0.0,0.0,0.004609,0.001863,0.0,-0.00618,0.107091,0.993481,0.099313
3,,4,124,RectifierDropout,50.0,0.0,0.0,0.009128,0.013772,0.0,-0.04455,0.109691,0.882699,0.078754
4,,5,2,Softmax,,0.0,0.0,0.001545,0.000485,0.0,0.054346,0.413657,0.009328,0.076379




ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.22546240296963635
RMSE: 0.47482881438433827
LogLoss: 0.6409573659814459
Mean Per-Class Error: 0.42267851865391676
AUC: 0.694253131180271
AUCPR: 0.7022731837739116
Gini: 0.388506262360542

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.33578384677702056: 


,,0,1,Error,Rate
0,0,1157.0,3895.0,0.771,(3895.0/5052.0)
1,1,375.0,4667.0,0.0744,(375.0/5042.0)
2,Total,1532.0,8562.0,0.423,(4270.0/10094.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010040,0.745061,1.909698,1.909698,0.788382,0.769133,0.788382,0.769133,0.019173,0.019173,90.969765,90.969765,0.015554
1,2,0.020037,0.720565,1.937841,1.923740,0.800000,0.732193,0.794179,0.750701,0.019374,0.038547,93.784057,92.373985,0.031523
2,3,0.030035,0.701973,1.746075,1.864600,0.720833,0.710695,0.769764,0.737384,0.017457,0.056004,74.607509,86.460041,0.044227
3,4,0.040033,0.684082,1.746075,1.835000,0.720833,0.692666,0.757544,0.726216,0.017457,0.073461,74.607509,83.499991,0.056930
4,5,0.050031,0.669438,1.604774,1.788993,0.662500,0.677035,0.738551,0.716388,0.016044,0.089506,60.477422,78.899311,0.067228
5,6,0.100021,0.599071,1.552291,1.670691,0.640833,0.634283,0.689713,0.675353,0.077598,0.167104,55.229104,67.069137,0.114248
6,7,0.150010,0.534666,1.400897,1.580785,0.578333,0.565468,0.652597,0.638734,0.070030,0.237134,40.089724,58.078497,0.148379
7,8,0.200000,0.485276,1.245466,1.496973,0.514167,0.508488,0.617996,0.606180,0.062260,0.299395,24.546628,49.697275,0.169277
8,9,0.300021,0.418478,1.126910,1.373601,0.465223,0.448523,0.567065,0.553620,0.112714,0.412109,12.690957,37.360123,0.190896
9,10,0.400000,0.381889,1.046636,1.291877,0.432083,0.397541,0.533326,0.514608,0.104642,0.516751,4.663576,29.187689,0.198837




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.504725,0.034948,0.474401,0.498116,0.518704,0.557816,0.474587
1,auc,0.641262,0.007956,0.640170,0.638191,0.648740,0.649119,0.630087
2,err,0.495275,0.034948,0.525599,0.501884,0.481296,0.442184,0.525413
3,err_count,2377.800000,168.603680,2546.000000,2398.000000,2303.000000,2130.000000,2512.000000
4,f0point5,0.497987,0.013128,0.484660,0.498291,0.501309,0.517896,0.487779
5,f1,0.592950,0.003212,0.588824,0.597381,0.594042,0.593356,0.591146
6,f2,0.733842,0.023633,0.750021,0.745662,0.728869,0.694556,0.750103
7,lift_top_group,1.911589,0.100806,1.940327,1.899841,1.930747,2.032552,1.754477
8,logloss,0.646928,0.003337,0.646516,0.648735,0.642797,0.645109,0.651482
9,max_per_class_error,0.754770,0.099097,0.833742,0.784356,0.717676,0.600212,0.837867



Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,...,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_r2,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2022-05-12 07:34:00,0.000 sec,None,0.000000,0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2022-05-12 07:34:08,25 min 16.744 sec,1260 obs/sec,0.051720,1,1457.0,0.563472,0.883441,-0.270002,...,0.555967,1.387513,0.498910,0.525199,0.780316,-0.130262,0.567588,0.487682,1.602126,0.556900
2,,2022-05-12 07:34:30,25 min 38.113 sec,1875 obs/sec,1.393099,27,39245.0,0.497296,0.687713,0.010786,...,0.631809,1.684837,0.490886,0.485717,0.664601,0.033286,0.615320,0.553999,2.136167,0.569653
3,,2022-05-12 07:34:47,25 min 54.754 sec,1986 obs/sec,2.528451,49,71229.0,0.501086,0.695590,-0.004350,...,0.649363,1.746829,0.470378,0.485902,0.664939,0.032548,0.624886,0.562552,1.678417,0.564421
4,,2022-05-12 07:35:04,26 min 12.023 sec,2025 obs/sec,3.705690,72,104393.0,0.490474,0.673757,0.037742,...,0.660473,1.783946,0.459778,0.479414,0.652044,0.058210,0.631510,0.569817,1.907292,0.549052
5,,2022-05-12 07:35:20,26 min 28.196 sec,2062 obs/sec,4.836144,94,136239.0,0.489282,0.671267,0.042413,...,0.669040,1.884220,0.489895,0.480507,0.654286,0.053914,0.635555,0.566494,1.754709,0.518640
6,,2022-05-12 07:35:37,26 min 45.602 sec,1971 obs/sec,5.710553,111,160872.0,0.482761,0.657652,0.067768,...,0.674950,1.783946,0.477908,0.478448,0.650027,0.062003,0.633576,0.570867,1.907292,0.548398
7,,2022-05-12 07:35:56,27 min 4.920 sec,1897 obs/sec,6.643073,129,187142.0,0.496176,0.685980,0.015238,...,0.680476,1.744302,0.439667,0.482767,0.658791,0.044993,0.636815,0.565571,1.754709,0.525180
8,,2022-05-12 07:36:19,27 min 27.513 sec,1844 obs/sec,7.822087,152,220356.0,0.480470,0.652883,0.076595,...,0.687945,1.744302,0.440559,0.478098,0.649351,0.063377,0.640472,0.570135,1.754709,0.507521
9,,2022-05-12 07:36:37,27 min 46.087 sec,1816 obs/sec,8.752476,170,246566.0,0.486000,0.664627,0.055214,...,0.694961,1.803767,0.423123,0.478672,0.650438,0.061124,0.641514,0.571254,1.525834,0.513734
